# 📊 Data Exploration and Visualization

**Project:** Deep Reinforcement Learning for Dynamic Asset Allocation  
**Notebook:** 01 - Data Exploration  
**Purpose:** Explore market data, visualize asset characteristics, and understand regime patterns

---

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("✅ Imports complete")

## 2. Load Data

In [ ]:
# Load complete dataset
data_path = '../data/processed/complete_dataset.csv'
df = pd.read_csv(data_path, index_col=0, parse_dates=True)

print(f"📅 Date range: {df.index[0]} to {df.index[-1]}")
print(f"📊 Shape: {df.shape}")
print(f"📈 Columns: {df.columns.tolist()}")

df.head()

## 3. Data Summary Statistics

In [ ]:
# Summary statistics
print("\n" + "="*80)
print("PRICE STATISTICS")
print("="*80)

price_cols = [col for col in df.columns if col.startswith('price_')]
df[price_cols].describe()

In [ ]:
print("\n" + "="*80)
print("RETURN STATISTICS")
print("="*80)

return_cols = [col for col in df.columns if col.startswith('return_')]
df[return_cols].describe()

## 4. Asset Price Evolution

In [ ]:
# Plot normalized prices (all start at 100)
fig, ax = plt.subplots(figsize=(14, 6))

for col in price_cols:
    normalized = 100 * df[col] / df[col].iloc[0]
    asset_name = col.replace('price_', '')
    ax.plot(df.index, normalized, label=asset_name, linewidth=2)

ax.set_title('Normalized Asset Prices (Base = 100)', fontsize=16, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Normalized Price', fontsize=12)
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)

# Highlight major events
ax.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.5, label='COVID-19 Crash')

plt.tight_layout()
plt.show()

# Calculate total returns
print("\n" + "="*80)
print("TOTAL RETURNS (Buy-and-Hold)")
print("="*80)

for col in price_cols:
    asset_name = col.replace('price_', '')
    total_return = (df[col].iloc[-1] / df[col].iloc[0] - 1) * 100
    print(f"{asset_name:8s}: {total_return:8.2f}%")

## 5. Return Distribution Analysis

In [ ]:
# Plot return distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, col in enumerate(return_cols):
    asset_name = col.replace('return_', '')
    
    # Histogram + KDE
    axes[i].hist(df[col].dropna(), bins=50, alpha=0.6, edgecolor='black', density=True)
    df[col].plot(kind='kde', ax=axes[i], color='red', linewidth=2)
    
    # Statistics
    mean = df[col].mean()
    std = df[col].std()
    skew = df[col].skew()
    kurt = df[col].kurtosis()
    
    axes[i].set_title(f'{asset_name} Daily Returns', fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Return', fontsize=10)
    axes[i].set_ylabel('Density', fontsize=10)
    
    # Add text box with stats
    stats_text = f'μ={mean:.4f}\nσ={std:.4f}\nSkew={skew:.2f}\nKurt={kurt:.2f}'
    axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes,
                 fontsize=9, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Correlation Analysis

In [ ]:
# Correlation matrix
corr_matrix = df[return_cols].corr()

# Rename for display
asset_names = [col.replace('return_', '') for col in return_cols]
corr_matrix.index = asset_names
corr_matrix.columns = asset_names

# Plot heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
ax.set_title('Asset Return Correlations', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n📊 Correlation Insights:")
print(f"   - SPY-TLT correlation: {corr_matrix.loc['SPY', 'TLT']:.3f} (flight-to-safety effect)")
print(f"   - SPY-GLD correlation: {corr_matrix.loc['SPY', 'GLD']:.3f} (gold as diversifier)")
print(f"   - SPY-BTC correlation: {corr_matrix.loc['SPY', 'BTC-USD']:.3f} (crypto vs traditional)")

## 7. Rolling Volatility

In [ ]:
# Calculate rolling 30-day volatility (annualized)
window = 30
rolling_vol = df[return_cols].rolling(window=window).std() * np.sqrt(252)

# Plot
fig, ax = plt.subplots(figsize=(14, 6))

for col in return_cols:
    asset_name = col.replace('return_', '')
    ax.plot(df.index, rolling_vol[col], label=asset_name, linewidth=1.5)

ax.set_title(f'{window}-Day Rolling Volatility (Annualized)', fontsize=16, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Volatility (annualized)', fontsize=12)
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)

# Highlight COVID-19 period
ax.axvspan(pd.Timestamp('2020-02-01'), pd.Timestamp('2020-04-30'), 
           alpha=0.2, color='red', label='COVID-19 Volatility Spike')

plt.tight_layout()
plt.show()

## 8. Market Regime Analysis

In [ ]:
# Check if regime data exists
regime_col = None
for col in ['regime_gmm', 'regime_hmm', 'regime']:
    if col in df.columns:
        regime_col = col
        break

if regime_col:
    print(f"✅ Found regime column: {regime_col}\n")
    
    # Regime distribution
    regime_counts = df[regime_col].value_counts().sort_index()
    print("Regime Distribution:")
    print(regime_counts)
    print(f"\nRegime proportions:")
    print(regime_counts / len(df) * 100)
    
    # Visualize regimes over time
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Panel 1: SPY price with regime background
    ax1 = axes[0]
    ax1.plot(df.index, df['price_SPY'], color='black', linewidth=1.5, label='SPY Price')
    
    # Color background by regime
    regime_colors = {0: 'red', 1: 'yellow', 2: 'green'}
    regime_labels = {0: 'Bear/Volatile', 1: 'Transitional', 2: 'Bull'}
    
    for regime in [0, 1, 2]:
        regime_mask = df[regime_col] == regime
        regime_dates = df.index[regime_mask]
        
        for date in regime_dates:
            ax1.axvspan(date, date + pd.Timedelta(days=1), 
                       alpha=0.2, color=regime_colors[regime])
    
    ax1.set_title('SPY Price with Market Regimes', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Price ($)', fontsize=12)
    ax1.legend(loc='upper left', fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Panel 2: Regime over time
    ax2 = axes[1]
    ax2.plot(df.index, df[regime_col], color='blue', linewidth=1, alpha=0.7)
    ax2.fill_between(df.index, df[regime_col], alpha=0.3)
    ax2.set_title('Market Regime Classification', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Date', fontsize=12)
    ax2.set_ylabel('Regime', fontsize=12)
    ax2.set_yticks([0, 1, 2])
    ax2.set_yticklabels(['Bear/Volatile', 'Transitional', 'Bull'])
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Regime-conditional statistics
    print("\n" + "="*80)
    print("REGIME-CONDITIONAL RETURNS (SPY)")
    print("="*80)
    
    for regime in [0, 1, 2]:
        regime_mask = df[regime_col] == regime
        regime_returns = df.loc[regime_mask, 'return_SPY']
        
        print(f"\nRegime {regime} ({regime_labels[regime]}):")
        print(f"  Mean return: {regime_returns.mean():.6f} ({regime_returns.mean()*252*100:.2f}% annualized)")
        print(f"  Volatility:  {regime_returns.std():.6f} ({regime_returns.std()*np.sqrt(252)*100:.2f}% annualized)")
        print(f"  Days: {len(regime_returns)}")
else:
    print("⚠️ No regime data found in dataset")

## 9. VIX and Macro Indicators

In [ ]:
# Plot VIX if available
if 'VIX' in df.columns:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Panel 1: SPY vs VIX
    ax1 = axes[0]
    ax1_twin = ax1.twinx()
    
    ax1.plot(df.index, df['price_SPY'], color='blue', linewidth=1.5, label='SPY')
    ax1_twin.plot(df.index, df['VIX'], color='red', linewidth=1.5, label='VIX', alpha=0.7)
    
    ax1.set_ylabel('SPY Price ($)', fontsize=12, color='blue')
    ax1_twin.set_ylabel('VIX Level', fontsize=12, color='red')
    ax1.set_title('SPY Price vs VIX (Fear Index)', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Panel 2: VIX histogram
    ax2 = axes[1]
    ax2.hist(df['VIX'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='red')
    ax2.axvline(df['VIX'].mean(), color='blue', linestyle='--', linewidth=2, label=f'Mean: {df["VIX"].mean():.2f}')
    ax2.axvline(30, color='orange', linestyle='--', linewidth=2, label='High Vol Threshold (30)')
    ax2.set_title('VIX Distribution', fontsize=14, fontweight='bold')
    ax2.set_xlabel('VIX Level', fontsize=12)
    ax2.set_ylabel('Frequency', fontsize=12)
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 VIX Statistics:")
    print(f"   Mean: {df['VIX'].mean():.2f}")
    print(f"   Std:  {df['VIX'].std():.2f}")
    print(f"   Max:  {df['VIX'].max():.2f} (reached on {df['VIX'].idxmax()})")
    print(f"   Days with VIX > 30: {(df['VIX'] > 30).sum()} ({(df['VIX'] > 30).sum() / len(df) * 100:.1f}%)")
else:
    print("⚠️ VIX data not found")

## 10. Summary Statistics Table

In [ ]:
# Create comprehensive summary table
summary_stats = []

for i, ret_col in enumerate(return_cols):
    asset_name = ret_col.replace('return_', '')
    price_col = f'price_{asset_name}'
    
    returns = df[ret_col].dropna()
    
    # Calculate metrics
    total_return = (df[price_col].iloc[-1] / df[price_col].iloc[0] - 1) * 100
    annual_return = returns.mean() * 252 * 100
    annual_vol = returns.std() * np.sqrt(252) * 100
    sharpe = (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() > 0 else 0
    
    # Downside metrics
    downside_returns = returns[returns < 0]
    downside_vol = downside_returns.std() * np.sqrt(252) * 100 if len(downside_returns) > 0 else 0
    sortino = (returns.mean() / downside_returns.std()) * np.sqrt(252) if len(downside_returns) > 0 and downside_returns.std() > 0 else 0
    
    # Drawdown
    cum_returns = (1 + returns).cumprod()
    running_max = cum_returns.expanding().max()
    drawdown = (cum_returns - running_max) / running_max
    max_dd = drawdown.min() * 100
    
    summary_stats.append({
        'Asset': asset_name,
        'Total Return (%)': total_return,
        'Annual Return (%)': annual_return,
        'Volatility (%)': annual_vol,
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'Max Drawdown (%)': max_dd,
        'Skewness': returns.skew(),
        'Kurtosis': returns.kurtosis()
    })

summary_df = pd.DataFrame(summary_stats)
summary_df = summary_df.sort_values('Sharpe Ratio', ascending=False)

print("\n" + "="*100)
print("COMPREHENSIVE ASSET STATISTICS (2014-2024)")
print("="*100)
print(summary_df.to_string(index=False))
print("="*100)

## 11. Conclusions

### Key Insights:

1. **Asset Characteristics:**
   - **SPY (S&P 500)**: Strong returns, moderate volatility, positive skew
   - **TLT (Bonds)**: Lower returns, lower volatility, flight-to-safety behavior
   - **GLD (Gold)**: Diversification benefits, negative correlation with equities during stress
   - **BTC-USD**: Highest returns and volatility, emerging asset class

2. **Correlations:**
   - SPY and TLT show negative correlation (classic stock-bond diversification)
   - Gold provides uncorrelated returns (portfolio hedge)
   - Bitcoin correlation with traditional assets varies over time

3. **Market Regimes:**
   - Clear regime shifts visible (bull, bear, transitional)
   - Regime detection captures major market events (2020 COVID, etc.)
   - Return distributions differ significantly across regimes

4. **Volatility Patterns:**
   - Volatility clustering is evident (GARCH effects)
   - Major spikes during crisis periods (2020 COVID)
   - VIX inversely correlated with SPY (fear gauge)

5. **Implications for RL:**
   - Time-varying correlations justify adaptive allocation
   - Regime information should improve agent performance
   - Risk-adjusted returns (Sharpe) vary significantly by asset
   - Fat tails and skewness require robust risk management

---

**Next Steps:**
- Analyze baseline strategy performance (Notebook 02)
- Train RL agents with regime-augmented state (Scripts)
- Compare RL vs classical strategies (Notebook 04)


In [ ]:
# Save summary table
output_path = '../docs/data_summary_statistics.csv'
summary_df.to_csv(output_path, index=False)
print(f"\n✅ Summary statistics saved to {output_path}")